In [26]:
from spacy.lang.en import English
import pandas as pd
 
def match_entities(text):
    df = pd.read_csv('../../data/stock/stock.csv')
 
    symbol_data_caps = [{"label": 'SYM', "pattern": d['Symbol']} for d in df.iloc]
    symbol_data_lower = [{"label": 'SYM', "pattern": d['Symbol'].lower()} for d in df.iloc]
    org_data_caps = [{"label": 'ORG', "pattern": d['Company Name']} for d in df.iloc]
    org_data_lower = [{"label": 'ORG', "pattern": d['Company Name'].lower()} for d in df.iloc]
    org_data2_caps = [{"label": 'ORG', "pattern": d['Company Name'].split()[0]} for d in df.iloc]
    org_data2_lower = [{"label": 'ORG', "pattern": d['Company Name'].split()[0].lower()} for d in df.iloc]
    stock_patterns = symbol_data_caps + org_data_caps + org_data2_caps + symbol_data_lower + org_data_lower + org_data2_lower
 
    nlp = English()
    ruler = nlp.add_pipe("entity_ruler")
    ruler.add_patterns(stock_patterns)
 
    doc = nlp(text)
    result = [(ent.text, ent.label_) for ent in doc.ents]
    return result

In [27]:
def group_numerical_sequences(lst):
    lst.sort()  # Sort the list first
    result = []
    temp_sequence = [lst[0]]

    for i in range(1, len(lst)):
        if lst[i] - lst[i-1] == 1:
            print("Here")  # If the current number is consecutive to the previous one
            temp_sequence.append(lst[i])
        else:
            print('No Here')
            result.append(temp_sequence)  # Add the current sequence to the result
            temp_sequence = [lst[i]]  # Start a new sequence

    result.append(temp_sequence)  # Add the last sequence

    return result

In [28]:
text = "Apple Shares today"
def find_entities(text): 
    result = match_entities(text)
    text_list = text.split()
    total_entities = []
    entity_index = []
    if not result:
        total_entities = []
    else:
        entities = [ent[0] for ent in result]
        if len(entities) == 1:
            total_entities.append(entities[0])
        else:
            for idx,ent in enumerate(entities):
                if ent in text_list:
                    entity_index.append(text_list.index(ent))
            if not entity_index:
                total_entities = []
            elif len(entity_index) == 1:
                print('No here')
                total_entities.append(ent)
            else:
                entities_group = group_numerical_sequences(entity_index)
                for ent_grp in entities_group:
                    grouped_entity = [text_list[index] for index in ent_grp]
                    total_entities.append(' '.join(grouped_entity))
    
    return total_entities

In [29]:
df = pd.read_csv('../../data/stock/stock.csv')
df.head()

,Symbol,Company Name,Industry,Market Cap
0,TSLA,Tesla,Automotive,500B
1,MSFT,"""Microsoft""",Software,1000B
2,A,"Agilent Technologies, Inc.",Diagnostics & Research,42.25B
3,AA,Alcoa Corporation,Aluminum,6.50B
4,AACG,ATA Creativity Global,Education & Training Services,33.57M


In [41]:
stock_names = list(df['Company Name'].unique())
entities = find_entities("tell me about twist")
print(stock_names)
print(entities)
# for ent in entities:


['Tesla', ' "Microsoft"', 'Agilent Technologies, Inc.', 'Alcoa Corporation', 'ATA Creativity Global', 'Armada Acquisition Corp. I', 'Ares Acquisition Corporation II', 'Aadi Bioscience, Inc.', 'African Agriculture Holdings Inc.', 'American Airlines Group Inc.', 'Altisource Asset Management Corporation', 'Atlantic American Corporation', "The Aaron's Company, Inc.", 'Applied Optoelectronics, Inc.', 'AAON, Inc.', 'Advance Auto Parts, Inc.', 'Apple Inc', 'American Assets Trust, Inc.', 'AllianceBernstein Holding L.P.', 'American Battery Technology Company', 'AbbVie Inc.', 'Ameris Bancorp', 'AbCellera Biologics Inc.', 'Abeona Therapeutics Inc.', 'Ambev S.A.', 'Asbury Automotive Group, Inc.', 'ARCA biopharma, Inc.', 'Abacus Life, Inc.', 'Able View Inc.', 'ABM Industries Incorporated', 'Airbnb, Inc.', 'Acumen Pharmaceuticals, Inc.', 'Arbor Realty Trust, Inc.', 'Absci Corporation', 'Abbott Laboratories', 'Abits Group Inc.', 'Arbutus Biopharma Corporation', 'ABVC BioPharma, Inc.', 'Abivax SA', 'A

In [42]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def most_similar_text(input_text, text_list):
    vectorizer = CountVectorizer().fit_transform([input_text] + text_list)
    similarities = cosine_similarity(vectorizer)[0][1:]
    most_similar_index = similarities.argmax()
    return text_list[most_similar_index], similarities[most_similar_index]



most_similar, similarity_score = most_similar_text(entities[0], stock_names)
print("Most similar text:", most_similar)
print("Similarity score:", similarity_score)


Most similar text: Twist Bioscience Corporation
Similarity score: 0.5773502691896258
